The cell below defines functions to calculate the consumption path for CRRA utility.

In [ ]:
using Optim
using Plots
function u(c, ρ)
    if ρ==1
        return log(c)
    else
        return (c^(1-ρ)-1)/(1-ρ)
    end
end

function consumption(a0, y, m, β, δ, ρ)
    current_budget = a0 + y[1] -m[1]
    #println(length(y))
    if length(y) != length(m)
        return -Inf
    elseif length(y)==1
        return [current_budget]
    elseif current_budget <0
        return [-Inf]
    else
        next_y = y[2:end]
        next_m = m[2:end]
        next_c(c) = consumption(current_budget-c, next_y, next_m, β, δ, ρ)
        function utility(c)
            next_c_vec = next_c(c)
            if c<0 || minimum(next_c_vec) < 0
                return Inf
            end
            U = u(c, ρ)+ β * sum([δ^t * u(next_c_vec[t], ρ) for t in 1:length(next_c_vec)])
            return -U
        end
        if length(y)==5
            c_opt = optimize(utility, 0, current_budget, store_trace= true)
        else
            c_opt = optimize(utility, 0, current_budget)
        end
        c_star = Optim.minimizer(c_opt)
        c_vec =  next_c(c_star)
        pushfirst!(c_vec, c_star)
        return c_vec
    end
end